In [ ]:
# загрузка файла с гугл диска
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


### Загрузка необходимых модулей и настройка 💾

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import islice
import copy
import csv
import os

!git clone https://github.com/Evgenvg78/functions_for_piranha
!pip install moexalgo

import sys
sys.path.append('/content/functions_for_piranha')
from tvr_df import TVR_transform
from dataS import stat_df # модуль для обработки списка сделок
from tvr_df import TVR_asis # перобразует TVR в датафрейм без изменений

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

fatal: destination path 'functions_for_piranha' already exists and is not an empty directory.


### считываение данных

In [ ]:
# создаем ссылки на файлы
udin_tvr = '/content/drive/Othercomputers/Udin/robot_ 29.03.2021/Default.tvr2'
udin_variant = '/content/drive/MyDrive/work_data/TVR/var_group_ulia.txt'
udin_dir = '/content/drive/Othercomputers/Udin/robot_ 29.03.2021/my deals/2024*.txt'

artem_tvr = '/content/drive/Othercomputers/Artem/ROBOT/Default.tvr2'
artem_variant = '/content/drive/MyDrive/work_data/TVR/var_group.txt'
artem_dir = '/content/drive/Othercomputers/Artem/ROBOT/my deals/2024*.txt'

In [ ]:
def range_num_to_list(range_num):
  result = []
  for element in range_num.split(','):
    split_index = element.find('-')
    if split_index == -1:
      result.append(int(element))
    else:
      start = int(element[:split_index])
      end = int(element[split_index+1:])
      result.extend(list(range(start, end+1)))

  return result



In [ ]:
account = 'U' # если 'U', то смотрим счет Юдина, если 'A', то смотрим счет Артема
if account == 'A':
  tvr_file = artem_tvr
  variant = artem_variant
  dir = artem_dir
else:
  tvr_file = udin_tvr
  variant = udin_variant
  dir = udin_dir

TVR = TVR_asis(tvr_file)
# df = stat_df(dir, TVR, default_comis=0.25) # default_comis = указываем сколько копеек за контракт с нас дерет брокер.

### формирование словаря

In [ ]:
# Считывание первой строки и разделителя
with open(tvr_file, encoding='utf8') as file:
    first_line = file.readline()  # Считываем первую строку
    d = first_line[0]  # Получаем элемент, разделяющий названия колонок

# Считывание названий колонок
column_names = first_line.split(d)[2:]  # Убираем первые два элемента (это пробелы)

# Создаем словарь соответствия названий столбцов и их номеров
column_dict = {name: idx + 1 for idx, name in enumerate(column_names)}


### рабочая функция парсинга стркутуры

In [ ]:


def remove_nan_keys(my_dict):
    """
    Удаляет ключи с NaN значениями из словаря.
    """
    new_dict = {}
    for key, value in my_dict.items():
        try:
            if not np.isnan(float(value)):
                new_dict[key] = value
        except (TypeError, ValueError):
            # Если значение не является числом, сохраняем его как есть
            new_dict[key] = value
    return new_dict

def parse_strategy(df, strategy_num, base_stroka=None, visited=None):
    """
    Рекурсивно парсит строку стратегии и связанные с ней фильтры, используя базовую строку.

    :param df: DataFrame с настройками
    :param strategy_num: Значение в колонке 'stroka' для основной строки
    :param base_stroka: Номер базовой строки, относительно которой рассчитываются ссылки
    :param visited: Множество посещенных строк для предотвращения дублирования
    :return: Словарь с вложенными строками и фильтрами с относительными ссылками
    """
    if visited is None:
        visited = {}

    # Задаем базовую строку для относительных расчетов
    if base_stroka is None:
        base_stroka = strategy_num

    # Проверка на дублирование
    if strategy_num in visited:
        return visited[strategy_num]

    # Извлекаем строку стратегии
    filtered_df = df[df['stroka'] == strategy_num]
    if filtered_df.empty:
        raise ValueError(f"Строка с 'stroka' == {strategy_num} не найдена.")

    strategy_row = filtered_df.iloc[0]
    strategy_dict = strategy_row.to_dict()

    # Устанавливаем 'stroka' как относительное значение от base_stroka
    diff = strategy_num - base_stroka
    strategy_dict['stroka'] = f"begin+{diff}" if diff > 0 else f"begin{diff}"

    # Удаляем NaN ключи из строки стратегии
    # strategy_dict = remove_nan_keys(strategy_dict)
    strategy_dict['Pos'] = np.nan
    strategy_dict['S'] = np.nan
    strategy_dict['stroka'] = 'begin'  # !!!!! ВНЕТС для пробы
    strategy_dict = remove_nan_keys(strategy_dict)

    # Сохраняем текущую строку в visited, чтобы избежать дублирования
    visited[strategy_num] = strategy_dict

    # Список колонок с возможными ссылками на фильтры
    filter_columns = ['InL1', 'InL2', 'OutL1', 'OutL2']
    filters = {}

    for col in filter_columns:
        if pd.notna(strategy_dict.get(col)):
            filter_refs = str(strategy_dict[col]).split(',')
            transformed_refs = []
            for ref in filter_refs:
                ref = ref.strip()
                if ref.isdigit():
                    ref_num = int(ref)
                    relative_diff = ref_num - base_stroka
                    transformed_ref = f"begin+{relative_diff}" if relative_diff > 0 else f"begin{relative_diff}"
                    transformed_refs.append(transformed_ref)

                    # Рекурсивно парсим фильтр с относительными ссылками
                    if ref_num not in visited:
                        filters[transformed_ref] = parse_strategy(df, ref_num, base_stroka, visited)
                else:
                    transformed_refs.append(ref)
            # Обновляем значение в колонке на преобразованные ссылки
            strategy_dict[col] = ', '.join(transformed_refs)

    if filters:
        strategy_dict['filters'] = filters

    return strategy_dict


### Добавление шортовой строки

In [ ]:
def add_short_to_structure(base_structure):
  # Создаем копию начальной основной строки с изменениями
  additional_main_row = base_structure.copy()
  additional_main_row['stroka'] = 'begin+1'  # Новое значение stroka
  additional_main_row['Mode'] = -1           # Устанавливаем Mode на -1
  del additional_main_row['filters']             # Удаляем поле filters

  # Добавляем новую основную строку в структуру как новый фильтр с ключом 'start+1'
  base_structure['filters'] = base_structure.get('filters', {})
  base_structure['filters']['begin+1'] = additional_main_row

  # Теперь base_structure содержит основную строку и дополнительную строку с ключом 'start+1'

### рабочие функции переноса фильров:
- relocate_and_update_filter(structure, current_filter_key, new_filter_key)

In [ ]:
def relocate_and_update_filter(structure, current_filter_key, new_filter_key):
    """
    Переносит указанный фильтр в новую позицию, обновляет 'stroka' и ссылки на всех уровнях вложенности.

    :param structure: Вложенная структура стратегии
    :param current_filter_key: Текущий ключ (например, 'start+15') для фильтра
    :param new_filter_key: Новый ключ (например, 'start+3') для фильтра
    """

    def find_and_relocate_filter(struct):
        """
        Рекурсивно находит фильтр с current_filter_key, перемещает его на new_filter_key и обновляет 'stroka'.

        :param struct: Текущая структура для поиска
        :return: True, если фильтр найден и перемещен, иначе False
        """
        # Проверяем, есть ли 'filters' на текущем уровне
        if 'filters' in struct:
            # Проверяем, есть ли current_filter_key среди filters на этом уровне
            if current_filter_key in struct['filters']:
                # Переносим фильтр на новую позицию
                struct['filters'][new_filter_key] = struct['filters'].pop(current_filter_key)

                # Обновляем 'stroka' внутри перенесенного фильтра
                struct['filters'][new_filter_key]['stroka'] = new_filter_key

                # Возвращаем True, чтобы остановить дальнейшее рекурсивное выполнение
                return True

            # Рекурсивно ищем во вложенных фильтрах
            for _, nested_filter in struct['filters'].items():
                if find_and_relocate_filter(nested_filter):
                    return True
        return False

    def update_references(struct, old_key, new_key):
        """
        Рекурсивно обновляет ссылки с old_key на new_key во всех полях InL1, InL2, OutL1, OutL2.

        :param struct: Текущая структура для обновления ссылок
        """
        # Обновляем ссылки в текущей структуре (включая основную строку)
        for col in ['InL1', 'InL2', 'OutL1', 'OutL2']:
            if col in struct and struct[col] == old_key:
                struct[col] = new_key

        # Рекурсивно обновляем ссылки во вложенных фильтрах
        if 'filters' in struct:
            for _, filter_struct in struct['filters'].items():
                update_references(filter_struct, old_key, new_key)

    # Запускаем поиск и перемещение фильтра
    if find_and_relocate_filter(structure):
        # Если фильтр найден и перемещен, запускаем обновление ссылок
        update_references(structure, current_filter_key, new_filter_key)


In [ ]:
base = parse_strategy(TVR, 5562)
add_short_to_structure(base)
# relocate_and_update_filter(base, 'begin+10', 'begin+2')
# relocate_and_update_filter(base, 'begin+15', 'begin+3')
base

{'stroka': 'begin',
 'Start': 'True',
 'InL1': 'begin+8',
 'OutL1': 'begin+15',
 'OutL2': 'begin+12',
 'Sec 0': 'MEH5',
 'V 0': '3',
 'Sec 1': 'MEH5',
 'V 1': '0',
 'W 1': '1',
 '[a]': '0',
 '[b]': '0',
 '[n]': '0.001',
 'C': '2;600',
 'N': '0.001',
 'P': '-0.00001',
 'A': '1;10000',
 'C1': '1',
 'SM': '2;4;0.05;0.5',
 'Mode': '1',
 'St.D': '2024.08.26 07:00:00',
 'T.In': '09:00:10-18:50:10 19:05:10-23:49:30',
 'T.Out': '09:00:10-18:50:10 19:05:10-23:49:30',
 'NotSet': '0',
 'MoveN': 'm30/30',
 'filters': {'begin+8': {'stroka': 'begin',
   'InL1': 'begin+12',
   'Sec 0': 'MEH5',
   'V 0': '0',
   'Sec 1': 'MEH5',
   'V 1': '0',
   'W 1': '0',
   '[a]': '0',
   '[b]': '0',
   'C': '2;6',
   'N': '0.7',
   'P': '0.0001',
   'A': '2;60',
   'C1': '10',
   'SM': '1;1;0.001;0.1',
   'SrSv': '40;10000',
   'Mode': '1',
   'St.D': '2021.08.30 07:00:00',
   'T.In': '09:00:10-18:50:10 19:05:10-23:49:30',
   'T.Out': '09:00:10-18:50:10 19:05:10-23:49:30',
   'NotSet': '0',
   'filters': {'begin+

### функция внесения изменений в структуру

In [ ]:
def new_strategy_structure_N(base_structure, start, sec0, sec1, N_f,	P_f,	N,	P, N_sh,	P_sh, V0, W1, SM, begin):
  # start=stroka
  # strategy = base_structure.copy()
  strategy = copy.deepcopy(base_structure)

  # Устанавливаем значения для основной строки
  strategy['start'] = start # True/False
  strategy['Sec 0'] = sec0
  strategy['V 0'] = V0
  strategy['Sec 1'] = sec0
  # strategy['C'] = C
  strategy['N'] = N
  strategy['P'] = P
  strategy['SM'] = SM

  short = strategy['filters']['begin+1']
  short['start'] = start # True/False
  short['Sec 0'] = sec0
  short['V 0'] = V0
  short['Sec 1'] = sec0
  # short['C'] = C
  short['N'] = N_sh
  short['P'] = P_sh
  short['SM'] = SM


  # Явно прописываем значения для фильтров
  filtr1 = strategy['filters']['begin+2']
  filtr2 = filtr1['filters']['begin+3']
  filtr1['Sec 0'] = sec0
  filtr1['Sec 1'] = sec1
  filtr1['W 1'] = W1
  # filtr1['C'] = C
  filtr1['N'] = N_f
  filtr1['P'] = P_f



  filtr2['Sec 0'] = sec0
  filtr2['Sec 1'] = sec0



  def calculate_start(strategy, begin):
    keys_to_check = ['InL1', 'InL2', 'OutL1', 'OutL2', 'stroka']
    # start = stroka
    for key, value in strategy.items():
      if key in keys_to_check:
        # print(value)
        # Check if the value is a string potentially representing an expression
        if isinstance(value, str):
          try:
            # Attempt to evaluate the expression
            # If it's a simple arithmetic expression involving begin (list)
            # and an integer, it should work as intended after modification
            strategy[key] = eval(value, {'begin': begin})  # Pass begin as a local variable to eval
          except TypeError:
            # If TypeError occurs, it means the expression involves an
            # unsupported list concatenation with an int
            # Handle it by potentially converting the int to a list
            # or adjusting the expression, and retry evaluation
            if '+' in value and value.split('+')[0] == 'begin':
              try:
                offset = int(value.split('+')[1])
                strategy[key] = begin + [offset]  # Concatenate with a single-element list
              except (IndexError, ValueError):
                # If the expression is not in the expected format
                # or the offset is not an integer, skip or handle differently
                print(f"Warning: Could not evaluate expression '{value}' for key '{key}'")
            else:
              print(f"Warning: Could not evaluate expression '{value}' for key '{key}'")
        else:
          # If the value is not a string, keep it as is
          strategy[key] = value
      if isinstance(value, dict):
        calculate_start(value, begin)  # Pass begin as an argument
    return strategy

  calculate_start(strategy, begin)

  return strategy



#### проверка работы функции

In [ ]:
# df = new_strategy_structure_N(base,
#                              start=combined_df.loc[2, 'start'],
#                              sec0=combined_df.loc[2, 'CODE'],
#                              sec1=combined_df.loc[2, 'sec1'],
#                              C=combined_df.loc[2, 'C'],
#                              N=combined_df.loc[2, 'N'],
#                              P=combined_df.loc[2, 'P'],
#                              N_sh=combined_df.loc[2, 'N_sh'],
#                              V0=combined_df.loc[2, 'V0'],
#                              W1=combined_df.loc[2, 'W1'],
#                              SM=combined_df.loc[2, 'SM'],
#                              begin=combined_df.loc[2, 'stroka'])
# df

### Формирование датафрейма

In [ ]:
def structure_to_dataframe(strategy, column_mapping = column_dict):
    """
    Преобразует структуру стратегии со вложенными фильтрами в DataFrame.

    :param strategy: Структура стратегии со значениями
    :param column_mapping: Словарь с номерами столбцов для каждого ключа
    :return: DataFrame с полями 'stroka', 'stolbec' и 'data'
    """
    rows = []

    def add_rows(struct):
        # Извлекаем значение 'stroka' для текущей строки или фильтра
        stroka_value = struct.get('stroka', 'unknown')

        # Добавляем поля для текущей строки
        for key, value in struct.items():
            if key in column_mapping:
                rows.append({'stroka': stroka_value, 'stolbec': column_mapping[key], 'data': value})

        # Рекурсивно добавляем данные из вложенных фильтров
        if 'filters' in struct:
            for filter_key, filter_data in struct['filters'].items():
                add_rows(filter_data)

    # Запускаем добавление строк для основной строки и вложенных фильтров
    add_rows(strategy)

    return pd.DataFrame(rows)

### Код

In [ ]:
# Общее описание последовательности: 1) парсим строку из  TVR с помощью функции parse_strategy
# добавляем short строку (как фильр) с помощью функции add_short_to_structure
# если нужно то реллоцируем фильры relocate_and_update_filter (structure, current_filter_key, new_filter_key)
# через функцию new_strategy_struvrure прописываем что будем менять в стркутуре
# формируем датафрейм с помощью функции structure_to_dataframe
#
#

In [ ]:
sec_list = ['AFKS',
 'AFLT',
 'BANE',
 'FEES',
 'FLOT',
 'GMKN',
 'IRAO',
 'LKOH',
 'MAGN',
 'MGNT',
 'MOEX',
 'MTLR',
 'MVID',
 'NLMK',
 'NOTK',
 'PHOR',
 'PIKK',
 'PLZL',
 'RTKM',
 'RUAL',
 'SGZH',
 'SIBN',
 'SMLT',
 'SNGP',
 'SOFL',
 'TATN',
 'TCSI',
 'TRNF',
 'VKCO',
 'WUSH']

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Список вариантов для выпадающего окна
options = ['H5', 'M5', 'U5', 'Z5']

# Создание выпадающего окна
dropdown = widgets.Dropdown(
    options=options,
    value=options[0],  # Значение по умолчанию
    description='Выберите вариант:',
    disabled=False,
)

# Переменная, которой будет присвоено выбранное значение
last_digit = None

# Функция, которая будет вызвана при изменении выбора
def on_change(change):
    global last_digit
    last_digit = change['new']
    print(f'Вы выбрали: {ccc}')
    # Здесь можно продолжить выполнение кода

# Привязка функции к событию изменения значения
dropdown.observe(on_change, names='value')

# Отображение выпадающего окна
display(dropdown)

Dropdown(description='Выберите вариант:', options=('H5', 'M5', 'U5', 'Z5'), value='H5')

Вы выбрали: H5
Вы выбрали: H5


In [ ]:
mix_pattern

'MXH5'

In [ ]:
sec_new = pd.read_csv(r'/content/drive/MyDrive/work_data/TVR/sec_tvr_full.csv')
SEC_i = sec_new[['SECID', 'SHORTNAME', 'PREVSETTLEPRICE', 'INITIALMARGIN']]

SEC_i = SEC_i[SEC_i['SECID'].str.endswith(last_digits)]
mix_pattern = 'MX'+last_digit
SEC_i.rename(columns={'SECID': 'CODE', 'SHORTNAME': 'base_code', 'PREVSETTLEPRICE':'PRICE', 'INITIALMARGIN':'SELLDEPO'}, inplace=True)
MIX = SEC_i.loc[SEC_i['CODE']==mix_pattern].iloc[:,-2].item()
SEC_i['base_code'] = SEC_i['base_code'].apply(lambda x: x.split('-')[0])
SEC_i = SEC_i[SEC_i.base_code.isin(sec_list)]


SEC_i = SEC_i.reset_index(drop=True)


304650.0


In [ ]:
# params_num = 4

# params = pd.DataFrame({
#   'sec1':['MXZ4']*params_num,
#   'C':['10;60', '5;60', '20;60', '40;60'],
#   'N':[0.009, 0.006, 0.004, 0.007],
#   'P':[0.004, 0.001, 0.003, 0.002],
#   'N_sh':[0.01, 0.02, 0.03, 0.04],
#    'start':[False]*params_num,
#     'V0':[0]*params_num
# })
params = pd.read_csv('/content/drive/MyDrive/work_data/TVR/final_params.csv')
params = params[['con1_long', 'con2_long', 'con3_long', 'con7_long', 'con3',	'con7']]

a = len(params)
params['sec1'] = ['MXZ4']*a
params['start'] = [False]*a
params['V0'] = [0]*a
params.rename(columns={'con1_long':'N_f', 'con2_long':'P_f', 'con3_long':'N', 'con7_long':'P', 'con3': 'N_sh', 'con7': 'P_sh'}, inplace=True)
params.loc[params['P_f'] == 0, 'P_f'] = 0.0001
params.loc[params['P'] == 0, 'P'] = 0.0001
params.loc[params['P_sh'] == 0, 'P_sh'] = 0.0001
params

,N_f,P_f,N,P,N_sh,P_sh,sec1,start,V0
0,0.004,0.0001,0.006,0.0020,0.007,0.002,MXZ4,False,0
1,0.003,0.0001,0.005,0.0020,0.005,0.002,MXZ4,False,0
2,0.007,0.0020,0.004,0.0020,0.007,0.006,MXZ4,False,0
3,0.003,0.0040,0.006,0.0001,0.004,0.004,MXZ4,False,0
4,0.003,0.0020,0.006,0.0040,0.005,0.006,MXZ4,False,0
5,0.003,0.0040,0.005,0.0001,0.006,0.002,MXZ4,False,0
6,0.006,0.0020,0.005,0.0040,0.004,0.006,MXZ4,False,0
7,0.003,0.0020,0.004,0.0001,0.004,0.002,MXZ4,False,0
8,0.004,0.0001,0.003,0.0020,0.007,0.004,MXZ4,False,0
9,0.004,0.0001,0.004,0.0040,0.003,0.004,MXZ4,False,0


In [ ]:
print(len(SEC_i))
SEC_i = SEC_i[SEC_i['base_code'].isin(sec_list)]
print(len(SEC_i))

42
30


In [ ]:
strok_mezdu = 5
begin=7203
srok_v_skripte = 4




def main_data_to_fill(SEC_i, params, cap):

  SEC_i['key'] = 1
  params['key'] = 1
  combined_df = pd.merge(SEC_i, params, on='key').drop('key', axis=1)
  combined_df['W1'] = combined_df['PRICE']/MIX*1000000//1000/1000
  combined_df['SM'] = combined_df['PRICE'].apply(lambda x: f"{int(round(x/100 * 0.033, 0))};{int(round(x/100 * 0.1, 0))};0.01;0.5") # формируем колонку SM
  # cap = 700000  # Начальный капитал
  cap = cap*4

  sec_numb = combined_df['CODE'].nunique()  # Количество уникальных инструментов


  # Группируем по инструментам для распределения капитала
  grouped = combined_df.groupby('CODE')
  ini_allocated_cap = cap / sec_numb
  # print(ini_allocated_cap)
  total_used_cap = 0  # Переменная для подсчета использованного капитала
  used_cap_per_instrument = {}  # Словарь для хранения использованного капитала по каждому инструменту


  total_used_capGO = 0  # Переменная для подсчета использованного капитала
  used_cap_per_instrumentGO = {}  # Словарь для хранения использованного капитала по каждому инструменту

  for sec, group in grouped:
      allocated_cap = cap / sec_numb  # Капитал на каждый инструмент
      price_per_sec = group['PRICE'].iloc[0]
      used_cap = 0  # Переменная для подсчета использованного капитала на данный инструмент
      used_capGO = 0
      param_num = len(group)  # Количество параметров (максимальное)

      if allocated_cap < price_per_sec:  # Сколько используется параметров для данного инструмента
          param_used = 1
      elif allocated_cap / price_per_sec < param_num:
          param_used = int(allocated_cap // price_per_sec)
      else:
          param_used = param_num


      sec_num = 1 if allocated_cap < price_per_sec else int(allocated_cap // price_per_sec)  # Сколько штук sec используется
      print(f'планируется потратить {sec_num} на {sec}')
      base_value = sec_num // param_used  # Базовое распределение по параметрам
      params = [base_value] * param_used
      ostatok = sec_num % param_used
      for i in range(ostatok):
          params[i] += 1

      # Перебор ограниченного количества строк
      for idx, row in islice(group.iterrows(), param_used):
          combined_df.at[idx, 'V0'] = params.pop(0)
          combined_df.at[idx, 'start'] = True
          used_cap += combined_df.at[idx, 'V0'] * combined_df.at[idx, 'PRICE']
          used_capGO += combined_df.at[idx, 'V0'] * combined_df.at[idx, 'SELLDEPO']

  #     cap -= used_cap
      total_used_cap += used_cap
      total_used_capGO += used_capGO

      used_cap_per_instrument[sec] = used_cap  # Записываем использованный капитал для текущего инструмента
      used_cap_per_instrumentGO[sec] = used_capGO
  #     sec_numb -= 1

  print("Использованный капитал по каждому инструменту:", used_cap_per_instrument)
  print("Общий использованный капитал:", total_used_cap)
  combined_df = combined_df[['CODE', 'sec1', 'N_f',	'P_f',	'N',	'P', 'N_sh', 'P_sh', 'start', 'V0', 'W1', 'SM']]
  return combined_df



combined_df = main_data_to_fill(SEC_i, params, cap=500000)

# функция, чтобы добавить номера скриптов (начальные)
def stroki_v_skripte(combined_df, begin, srok_v_skripte, strok_mezdu):
  strok = srok_v_skripte+strok_mezdu
  combined_df['stroka'] = list(range(begin, begin+len(combined_df)*strok, strok))

  return combined_df

combined_df = stroki_v_skripte(combined_df, begin, srok_v_skripte, strok_mezdu)

планируется потратить 10 на AFZ4
планируется потратить 4 на AKZ4
планируется потратить 28 на BNZ4
планируется потратить 7 на FLZ4
планируется потратить 8 на FSZ4
планируется потратить 59 на GKZ4
планируется потратить 1 на IRZ4
планируется потратить 1 на LKZ4
планируется потратить 6 на MCZ4
планируется потратить 3 на MEZ4
планируется потратить 1 на MGZ4
планируется потратить 14 на MNZ4
планируется потратить 67 на MVZ4
планируется потратить 1 на NKZ4
планируется потратить 5 на NMZ4
планируется потратить 12 на PHZ4
планируется потратить 13 на PIZ4
планируется потратить 1 на PZZ4
планируется потратить 17 на RLZ4
планируется потратить 11 на RTZ4
планируется потратить 53 на S0Z4
планируется потратить 1 на SGZ4
планируется потратить 11 на SOZ4
планируется потратить 49 на SSZ4
планируется потратить 38 на SZZ4
планируется потратить 2 на TIZ4
планируется потратить 55 на TNZ4
планируется потратить 1 на TTZ4
планируется потратить 22 на VKZ4
планируется потратить 41 на WUZ4
Использованный капитал п

In [ ]:
combined_df.tail(10)

,CODE,sec1,N_f,P_f,N,P,N_sh,P_sh,start,V0,W1,SM,stroka
290,WUZ4,MXZ4,0.004,0.0001,0.006,0.0020,0.007,0.002,True,5,0.006,1;2;0.01;0.5,9813
291,WUZ4,MXZ4,0.003,0.0001,0.005,0.0020,0.005,0.002,True,4,0.006,1;2;0.01;0.5,9822
292,WUZ4,MXZ4,0.007,0.0020,0.004,0.0020,0.007,0.006,True,4,0.006,1;2;0.01;0.5,9831
293,WUZ4,MXZ4,0.003,0.0040,0.006,0.0001,0.004,0.004,True,4,0.006,1;2;0.01;0.5,9840
294,WUZ4,MXZ4,0.003,0.0020,0.006,0.0040,0.005,0.006,True,4,0.006,1;2;0.01;0.5,9849
295,WUZ4,MXZ4,0.003,0.0040,0.005,0.0001,0.006,0.002,True,4,0.006,1;2;0.01;0.5,9858
296,WUZ4,MXZ4,0.006,0.0020,0.005,0.0040,0.004,0.006,True,4,0.006,1;2;0.01;0.5,9867
297,WUZ4,MXZ4,0.003,0.0020,0.004,0.0001,0.004,0.002,True,4,0.006,1;2;0.01;0.5,9876
298,WUZ4,MXZ4,0.004,0.0001,0.003,0.0020,0.007,0.004,True,4,0.006,1;2;0.01;0.5,9885
299,WUZ4,MXZ4,0.004,0.0001,0.004,0.0040,0.003,0.004,True,4,0.006,1;2;0.01;0.5,9894


In [ ]:
def TVR_builder(TVR, combined_df):
  symb = TVR.iloc[0, 4]
  data_div = [symb]*45
  stroka_div = [begin-1]*45
  stolbec_div = list(range(1,46))
  div_dic = {'stroka': stroka_div, 'stolbec': stolbec_div, 'data':data_div}
  div_df = pd.DataFrame(div_dic) # разделитьель

  final_df = pd.DataFrame()

  final_df = pd.concat([final_df, div_df], ignore_index=True)
  for i in range(0, len(combined_df)):
    add_df = structure_to_dataframe(new_strategy_structure_N(base_structure = base,
                                                           start=combined_df.loc[i, 'start'],
                                                           sec0=combined_df.loc[i, 'CODE'],
                                                           sec1=combined_df.loc[i, 'sec1'],
                                                           N_f=combined_df.loc[i, 'N_f'],
                                                           P_f=combined_df.loc[i, 'P_f'],
                                                           N=combined_df.loc[i, 'N'],
                                                           P=combined_df.loc[i, 'P'],
                                                            N_sh=combined_df.loc[i, 'N_sh'],
                                                           P_sh=combined_df.loc[i, 'P_sh'],
                                                           V0=combined_df.loc[i, 'V0'],
                                                           W1=combined_df.loc[i, 'W1'],
                                                           SM=combined_df.loc[i, 'SM'],
                                                           begin=combined_df.loc[i, 'stroka']))

# (base_structure = base,
#                                                            start=combined_df.loc[i, 'start'],
#                                                            sec0=combined_df.loc[i, 'CODE'],
#                                                            sec1=combined_df.loc[i, 'sec1'],
#                                                            C=combined_df.loc[i, 'C'],
#                                                            N=combined_df.loc[i, 'N'],
#                                                            P=combined_df.loc[i, 'P'],
#                                                            N_sh=combined_df.loc[i, 'N_sh'],
#                                                            V0=combined_df.loc[i, 'V0'],
#                                                            W1=combined_df.loc[i, 'W1'],
#                                                            SM=combined_df.loc[i, 'SM'],
#                                                            begin=combined_df.loc[i, 'stroka']))

    if (final_df.loc[final_df['stolbec']==11].iloc[-1, 2] != combined_df.iloc[i, 0]) & (i !=0):
        a = final_df.iloc[-1, 0]

        # формирую разделитель
        data_div = [symb]*45
        stroka_div = [a+strok_mezdu]*45
        stolbec_div = list(range(1,46))
        div_dic = {'stroka': stroka_div, 'stolbec': stolbec_div, 'data':data_div}
        div_df = pd.DataFrame(div_dic) # разделитьель


        final_df = pd.concat([final_df, div_df], ignore_index=True)

    final_df = pd.concat([final_df, add_df], ignore_index=True)


  return final_df


final_DF = TVR_builder(TVR, combined_df)
print(final_DF.shape)

(25950, 3)


In [ ]:


# Заменить 'data.csv' на имя вашего файла
filename = '/content/drive/MyDrive/work_data/TVR/TVR_generate/pattern_data.csv'

# Открыть файл и сразу записать обновленное содержимое
with open(filename, 'w', encoding='utf-8') as f:
    # Добавить первую строку
    f.write(first_line + '\n')

    # Записать содержимое DataFrame без кавычек
    final_DF.to_csv(f, index=False, header=False, sep=' ', encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar=' ')

# Переименовать файл для другого формата
os.rename(filename, '/content/drive/MyDrive/work_data/TVR/TVR_generate/pattern_data_final.tvr')
